In [28]:
import tensorflow as tf  # For image preprocessing and model
from IPython.display import Image  # For displaying images in Jupyter
import os  # For file path manipulation
import tf_keras_vis

In [8]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [10]:
# List of image file paths
IMAGE_PATHS = ["Lenna.png"]

# Indices and layer name for Grad-CAM
indices = [263, 281]  # Class indices to visualize (e.g., from ImageNet)
layers_name = ['activation_6']  # Layer to focus on for Grad-CAM

In [29]:

for i in range(len(IMAGE_PATHS)):
    each_path = IMAGE_PATHS[i]  # Current image path
    index = indices[i]  # Target class index

    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(each_path, target_size=(224, 224))  # Resize to 224x224
    img = tf.keras.preprocessing.image.img_to_array(img)  # Convert to NumPy array
    img = tf.expand_dims(img, axis=0)  # Add batch dimension (shape becomes (1, 224, 224, 3))
    data = (img, None)  # Prepare data for Grad-CAM

    # Define the file name to save the visualization
    name = each_path.split("/")[-1].split(".jpg")[0]

    # Set up Grad-CAM
    # explainer = GradCAM()  # Instantiate Grad-CAM
    model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)  # Load pre-trained VGG16

    # Generate Grad-CAM heatmap
    tf_keras_vis.gradcam.GradCAM(model, "block5_conv3")
    #specified layer and class index

    # Save the Grad-CAM visualization
    # explainer.save(grid, '.', name + '_grad_cam.png')

    # Display the original image and Grad-CAM visualization
    # display_images([each_path, name + '_grad_cam.png'])


AttributeError: module 'tf_keras_vis' has no attribute 'gradcam'

In [21]:
data

AttributeError: 'list' object has no attribute 'shape'